In [2]:
import csv
import os

In [3]:
from pyproj import Transformer

In [4]:
fn = "/home/james/PycharmProjects/edinaseer/data/some_urls.csv"

if os.path.exists(fn):
    with open(fn, 'r') as inpf:
        my_reader = csv.DictReader(inpf)
        for r in my_reader:
            print(r["title"])
            print(r["url"])

St Albans City and District Council Amenity Green Space
http://inspirewms.stalbans.gov.uk/INSPIRE/WEBSERVICE/wms.exe?service=wms&request=GetCapabilities
Worcester City Council WMS Service
http://inspire.worcester.gov.uk/geoserver/worcester/ows?SERVICE=WMS&request=GetCapabilities
Wycombe Inspire Data
http://inspire.wycombe.gov.uk/getows.ashx?Mapsource=Wycombe/Inspire&service=WMS&version=1.1.1&Request=GetCapabilities
Fife Cycle Network
http://arcgisweb.fife.gov.uk/geoserver/fife/ows?request=getCapabilities&version=1.3.0&service=WMS
Suitable Alternative Natural Green Spaces
https://maps.runnymede.gov.uk/geoserver/environment/suitable_alternative_natural_green_spaces/wms?service=WMS&request=GetCapabilities
Amenity Greenspace
http://maps.cheshire.gov.uk/ArcGIS/services/CE/public_wms/MapServer/WmSServer?request=GetCapabilities&service=WMS
Amenity Greenspace Buffer
http://maps.cheshire.gov.uk/ArcGIS/services/CE/public_wms/MapServer/WmSServer?request=GetCapabilities&service=WMS
Amenity Greensp

In [24]:
my_url = 'http://inspirewms.stalbans.gov.uk/INSPIRE/WEBSERVICE/wms.exe?service=wms&request=GetCapabilities'


In [25]:
from owslib.wms import WebMapService

In [26]:
wms = WebMapService(my_url)

In [27]:
wms.version

'1.1.1'

In [28]:
wms.identification.title

'Cadcorp GeognoSIS OGC Web Map Service'

In [29]:
wms.identification.abstract

'OGC Web Map Service developed by Cadcorp'

In [30]:
list(wms.contents)

['Allotments',
 'Allotments_and_Community_Gardens',
 'Amenity_Green_Space',
 'Amenity_Green_Space.1',
 'Article_4_Areas',
 'Article_4_Areas.1',
 'Hedgerow_Retention_Notices',
 'Hedgerow_Retention_Notices.1',
 'Natural_Green_Space',
 'Natural_and_Semi-Natural_Green_Space',
 'Parks_and_Gardens',
 'Parks_and_Gardens.1',
 'Play_Areas',
 'Play_Areas.1',
 'Polling_Districts',
 'Polling_Districts.1',
 'Polling_Stations',
 'Polling_Stations.1',
 'S106',
 'Section_106_Agreements',
 'TPO',
 'Tree_Preservation_Orders']

In [31]:
wms['Polling_Districts'].title

'Polling_Districts'

In [32]:
[op.name for op in wms.operations]

['GetCapabilities', 'GetMap', 'GetFeatureInfo', 'DescribeLayer']

In [33]:
wms['Polling_Districts'].styles

{}

In [34]:
wms['Polling_Districts'].crsOptions

['EPSG:4277',
 'AUTO:42002',
 'EPSG:3857',
 'AUTO:42005',
 'EPSG:4326',
 'AUTO:42003',
 'AUTO:42001',
 'AUTO:42004',
 'EPSG:27700']

In [35]:
wms['Polling_Districts'].boundingBox

(-49416.8408732319,
 6743500.604849,
 -26438.5631655145,
 6773230.18568662,
 'EPSG:3857')

In [36]:
wms['Polling_Districts'].queryable

1

In [37]:
wms.getOperationByName('GetMap').formatOptions

['application/pdf',
 'application/vnd.ms-xpsdocument',
 'application/x-msmetafile',
 'application/x-postscript',
 'image/bmp',
 'image/gif',
 'image/jpeg',
 'image/jpeg; Quality=50',
 'image/jpeg; Quality=75',
 'image/jpeg; Quality=90',
 'image/jpeg; Quality=95',
 'image/png',
 'image/png; PhotometricInterpretation=PaletteColor',
 'image/png; PhotometricInterpretation=RGB',
 'image/png; PhotometricInterpretation=RGBA',
 'image/svg+xml',
 'image/x-webp',
 'image/x-webp; Quality=50',
 'image/x-webp; Quality=75',
 'image/x-webp; Quality=90',
 'image/x-webp; Quality=95']

In [88]:
def bbox_to_bng(src_crs, bbox, sub_bbox=False, buffer=100):
    bbox_bng = None
    if src_crs in (3857, 4326):
        src_x_min, src_y_min, src_x_max, src_y_max = bbox[0], bbox[1], bbox[2], bbox[3]
        transformer = Transformer.from_crs(src_crs, 27700)
        bng_xy_min = transformer.transform(src_x_min, src_y_min)
        bng_xy_max = transformer.transform(src_x_max, src_y_max)
        bbox_bng = (int(bng_xy_min[0]), int(bng_xy_min[1]), int(bng_xy_max[0]), int(bng_xy_max[1]))
        if sub_bbox:
            centroid_x = (bng_xy_max[0] + bng_xy_min[0]) / 2.0
            centroid_y = (bng_xy_max[1] + bng_xy_min[1]) / 2.0
            bbox_bng = (int(centroid_x), int(centroid_y), int(centroid_x+buffer), int(centroid_y+buffer))
            
    return bbox_bng        

In [89]:
bb = bbox = wms['Polling_Districts'].boundingBox
bb_bng = bbox_to_bng(3857, bb)
print(bb_bng)

(507669, 199698, 521493, 218402)


In [98]:
bb_bng = bbox_to_bng(3857, bb, sub_bbox=True, buffer=5000)

In [100]:
out_fn = '/home/james/Desktop/wms_map.png'

with open(out_fn, 'wb') as outpf:
    img = wms.getmap(
        layers=['Polling_Districts'],
        srs='EPSG:27700',
        bbox=bb_bng,
        size=(400,400),
        format='image/png'
        )
    outpf.write(img.read())
